In [13]:
import torch
from torchvision import datasets, transforms, models
from torch import optim
from torch.utils.data import dataloader
from torch.optim import lr_scheduler
import os
import copy
import time

In [11]:
# 加载数据

# 定义transform 函数
data_transform = {'train': transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])}

# 通过torchvision的datasets 导入图片，加载成torch的dataset格式
img_path = "./data/beef"
img_datasets = {x: datasets.ImageFolder(os.path.join(img_path,x),data_transform[x]) for x in ['train','val']}
# 加载成dataloader对图片进行batch 打包成一个迭代器
dataloaders = {x: dataloader.DataLoader(img_datasets[x],
                                        batch_size=4,
                                        shuffle=True,
                                        num_workers=4) 
                                        for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_size = {x: len(img_datasets[x]) for x in ['train', 'val']}


In [32]:
# 定义训练通用函数
def train_model(model, creterion, optimizer, scheduler, epoch_num=10):
    
    since = time.time()
    # 模型参数的保存
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(epoch_num):
        print("*"*20)
        print("Epoch {}/{}  training:".format(epoch + 1, epoch_num))


        for phase in ['train', 'val']:
            # 设置模型的模式：训练or验证
            if phase == 'train':
                model.train()
            else:
                model.eval()

            
            running_loss = 0.0
            running_acc = 0
            for inputs, targets in dataloaders[phase]:
                # 设置数据到指定的硬件设备：cpu or gpu
                inputs = inputs.to(device)
                targets = targets.to(device)

                # 梯度参数置零
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # 获得模型的预测值的输出，这里好像没有用softmax, 因为是2为输出
                    outputs = model(inputs)
                    # 查找每一行输入中最大值的那个
                    _, preds = torch.max(outputs, 1)
                    # 用交叉熵计算损失值
                    loss = creterion(outputs, targets)
                    
                    if phase == 'train':
                        # 计算反向传播的梯度值值
                        loss.backward()
                        # 更新权重参数
                        optimizer.step()

                # 计算一个batch的损失函数
                running_loss += loss.item() * inputs.size(0)
                running_acc += torch.sum(preds == targets)
            
            # 更新学习率
            if phase == 'train':
                scheduler.step()
            
            # 计算每个epoch的损失值与准确率
            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_acc / dataset_size[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc
            ))

            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

                    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [28]:
# 与训练模型的设置
model_tf = models.resnet18(pretrained=True)

# 设置模型为
for param in model_tf.parameters():
    param.requires_grad = False

num_fts = model_tf.fc.in_features

model_tf.fc = torch.nn.Linear(num_fts, 2)

model_tf.to(device)

creterion_ft = torch.nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_tf.fc.parameters(), lr=0.0001, momentum=0.9)

scheduler_ft = lr_scheduler.StepLR(optimizer=optimizer_ft, step_size=5, gamma=0.1)

In [33]:
model_trained = train_model(model_tf, creterion_ft, optimizer_ft, scheduler_ft)

********************
Epoch 1/10  training:
train Loss: 0.4156 Acc: 0.8279
val Loss: 0.2847 Acc: 0.9346
********************
Epoch 2/10  training:
train Loss: 0.4527 Acc: 0.7869
val Loss: 0.2729 Acc: 0.9150
********************
Epoch 3/10  training:
train Loss: 0.4102 Acc: 0.8484
val Loss: 0.2823 Acc: 0.9216
********************
Epoch 4/10  training:
train Loss: 0.3963 Acc: 0.8525
val Loss: 0.2783 Acc: 0.9281
********************
Epoch 5/10  training:
train Loss: 0.4730 Acc: 0.7828
val Loss: 0.2763 Acc: 0.9216
********************
Epoch 6/10  training:
train Loss: 0.4580 Acc: 0.7951
val Loss: 0.2813 Acc: 0.9216
********************
Epoch 7/10  training:
train Loss: 0.4527 Acc: 0.7951
val Loss: 0.2663 Acc: 0.9281
********************
Epoch 8/10  training:
train Loss: 0.4151 Acc: 0.8279
val Loss: 0.2764 Acc: 0.9281
********************
Epoch 9/10  training:
train Loss: 0.4080 Acc: 0.8443
val Loss: 0.2859 Acc: 0.9281
********************
Epoch 10/10  training:
train Loss: 0.4004 Acc: 0.836